# Analyze the co-occurences of the terms in the required qualifications

In [5]:
import os
import json
import string
import nltk
from nltk import word_tokenize, FreqDist, pos_tag
from nltk.corpus import stopwords
import pandas as pd
import pickle


In [2]:
df_ori = pd.read_csv('../data/data job posts.csv')
df_ori.head()
print(df_ori.shape)
df = df_ori.drop_duplicates(['RequiredQual'])
print(df.shape)
print("Removed {0} duplicates (based on RequiredQual)".format(df_ori.shape[0]-df.shape[0]))

(19001, 24)
(16689, 24)
Removed 2312 duplicates (based on RequiredQual)


In [3]:
df["RequiredQual"].head()

0    To perform this job successfully, an\r\nindivi...
1    - Bachelor's Degree; Master's is preferred;\r\...
2    - Degree in environmentally related field, or ...
3    - Advanced degree in public health, social sci...
4    - University degree; economical background is ...
Name: RequiredQual, dtype: object

In [4]:
df["RequiredQual"] = df["RequiredQual"].astype(str)

G:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Try the NLTK POS tagger

In [7]:
sent = '''Professor Tan Eng Chye, NUS Deputy President and Provost, and Professor 
Menahem Ben-Sasson, President of HUJ signed the joint degree agreement at NUS, 
in the presence of Ambassador of Israel to Singapore Her Excellency Amira Arnon 
and about 30 invited guests, on July 03, 2013.
'''
pos_tagged = pos_tag(word_tokenize(sent), tagset='universal')
#pos_tagged

In [8]:
df['RequiredQual_token'] = df['RequiredQual'].map(word_tokenize)
df['Required_Qual_POS'] = df['RequiredQual_token'].map(pos_tag, {"tagset": "universal"})

G:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
G:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
df['Required_Qual_POS'].head()

0    [(To, TO), (perform, VB), (this, DT), (job, NN...
1    [(-, :), (Bachelor, NN), ('s, POS), (Degree, N...
2    [(-, :), (Degree, NN), (in, IN), (environmenta...
3    [(-, :), (Advanced, VBD), (degree, JJ), (in, I...
4    [(-, :), (University, NNP), (degree, VBP), (;,...
Name: Required_Qual_POS, dtype: object

In [11]:
df['Required_Qual_POS'].to_csv("../outputs/nltk_pos1.csv", index=False)